## NFL Fantasy Rookie Project Part 1 - Data Scraping and Cleaning

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
years = list(range(2017,2023))

### Scraping

We are focusing part 1 on pulling data for both rushing and receiving as I will be looking into running backs (Part 1) and receivers (Part 2).

In [4]:
receiving_url = 'https://www.pro-football-reference.com/years/{}/receiving.htm'
rushing_url = 'https://www.pro-football-reference.com/years/{}/rushing.htm'

In [6]:
for year in years:
    url = receiving_url.format(year)
    data = requests.get(url)
    
    with open("C:/Users/kevin/Documents/receiving stats/{}.html".format(year), "w+", encoding='utf-8') as f:
        f.write(data.text)

In [7]:
for year in years:
    url = rushing_url.format(year)
    data = requests.get(url)
    
    with open("C:/Users/kevin/Documents/rushing stats/{}.html".format(year), "w+", encoding='utf-8') as f:
        f.write(data.text)

In [66]:
receiving = []
for year in years:
    with open("C:/Users/kevin/Documents/receiving stats/{}.html".format(year), encoding='utf-8') as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    regularstats = soup.find(id="receiving")
    regular = pd.read_html(str(regularstats))[0]
    regular["Year"] = year
    
    receiving.append(regular)

In [72]:
rushing = []
for year in years:
    with open("C:/Users/kevin/Documents/rushing stats/{}.html".format(year), encoding='utf-8') as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    regularstats = soup.find(id="rushing")
    regular = pd.read_html(str(regularstats))[0]
    regular["Year"] = year
    
    rushing.append(regular)

In [67]:
receiving = pd.concat(receiving)

In [73]:
rushing = pd.concat(rushing)

In [70]:
receiving.columns

Index(['Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'Tgt', 'Rec', 'Ctch%', 'Yds',
       'Y/R', 'TD', '1D', 'Lng', 'Y/Tgt', 'R/G', 'Y/G', 'Fmb', 'Year'],
      dtype='object')

In [21]:
rushing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2168 entries, 0 to 377
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   (Unnamed: 0_level_0, Rk)      2168 non-null   object
 1   (Unnamed: 1_level_0, Player)  2168 non-null   object
 2   (Unnamed: 2_level_0, Tm)      2168 non-null   object
 3   (Unnamed: 3_level_0, Age)     2168 non-null   object
 4   (Unnamed: 4_level_0, Pos)     2159 non-null   object
 5   (Games, G)                    2168 non-null   object
 6   (Games, GS)                   2168 non-null   object
 7   (Rushing, Att)                2168 non-null   object
 8   (Rushing, Yds)                2168 non-null   object
 9   (Rushing, TD)                 2168 non-null   object
 10  (Rushing, 1D)                 2168 non-null   object
 11  (Rushing, Lng)                2168 non-null   object
 12  (Rushing, Y/A)                2168 non-null   object
 13  (Rushing, Y/G)     

In [74]:
columns = ['Rk', 'Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'Rush_Att', 'Rush_Yds', 'Rush_TD', 'Rush_1D',
       'Rush_Lng', 'Rush_Y/A', 'Rush_Y/G', 'Rush_Fmb', 'Year']

In [75]:
rushing.columns = columns

### Data Cleaning and Manipulation

The sample from the dataframe shows that we will need to do some cleaning. For example, some players' names has * or +, which would make it impossible to look up stats later by name.

In [76]:
rushing.head()

,Rk,Player,Tm,Age,Pos,G,GS,Rush_Att,Rush_Yds,Rush_TD,Rush_1D,Rush_Lng,Rush_Y/A,Rush_Y/G,Rush_Fmb,Year
0,1,Le'Veon Bell*+,PIT,25,RB,15,15,321,1291,9,71,27,4.0,86.1,3,2017
1,2,LeSean McCoy*,BUF,29,RB,16,16,287,1138,6,57,48,4.0,71.1,3,2017
2,3,Melvin Gordon,LAC,24,RB,16,16,284,1105,8,61,87,3.9,69.1,1,2017
3,4,Todd Gurley*+,LAR,23,RB,15,15,279,1305,13,69,57,4.7,87.0,5,2017
4,5,Jordan Howard,CHI,23,RB,16,16,276,1122,9,65,53,4.1,70.1,1,2017


In [79]:
rushing.columns

Index(['Rk', 'Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'Rush_Att', 'Rush_Yds',
       'Rush_TD', 'Rush_1D', 'Rush_Lng', 'Rush_Y/A', 'Rush_Y/G', 'Rush_Fmb',
       'Year'],
      dtype='object')

In [81]:
receiving["Player"] = receiving["Player"].str.replace("*","", regex=False)
receiving["Player"] = receiving["Player"].str.replace("+","", regex=False)
rushing["Player"] = rushing["Player"].str.replace("*","", regex=False)
rushing["Player"] = rushing["Player"].str.replace("+","", regex=False)

In [82]:
def single_team(df):
    if df.shape[0]==1:
        return df
    else:
        row = df[df["Tm"]=="TOT"]
        row["Tm"] = df.iloc[-1,:]["Tm"]
        return row

receiving = receiving.groupby(["Player", "Year"]).apply(single_team)
rushing = rushing.groupby(["Player", "Year"]).apply(single_team)

In [83]:
receiving.index = receiving.index.droplevel()
receiving.index = receiving.index.droplevel()

rushing.index = rushing.index.droplevel()
rushing.index = rushing.index.droplevel()

In [84]:
receiving.to_csv('C:/Users/kevin/Downloads/All Receiving Stats 2017-2022.csv', index=False)
rushing.to_csv('C:/Users/kevin/Downloads/All Rushing Stats 2017-2022.csv', index=False)

The cleaning is complete. I am now creating duplicates of these statistical dataframes because a part of my model will include the stats from the rookie's best teammate. Because of that I will be pulling from the rushing and receiving dataframes twice: once for the rookie's stats and once for the teammate's stats.

In [85]:
teammate_receiving = receiving.copy()
teammate_rushing = rushing.copy()

In [86]:
receiving.columns

Index(['Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'Tgt', 'Rec', 'Ctch%', 'Yds',
       'Y/R', 'TD', '1D', 'Lng', 'Y/Tgt', 'R/G', 'Y/G', 'Fmb', 'Year'],
      dtype='object')

In [87]:
rushing.columns

Index(['Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'Rush_Att', 'Rush_Yds',
       'Rush_TD', 'Rush_1D', 'Rush_Lng', 'Rush_Y/A', 'Rush_Y/G', 'Rush_Fmb',
       'Year'],
      dtype='object')

In [88]:
teammate_receiving.columns = 'Teammate ' + teammate_receiving.columns
teammate_rushing.columns = 'Teammate ' + teammate_rushing.columns

In [89]:
teammate_receiving.columns

Index(['Teammate Player', 'Teammate Tm', 'Teammate Age', 'Teammate Pos',
       'Teammate G', 'Teammate GS', 'Teammate Tgt', 'Teammate Rec',
       'Teammate Ctch%', 'Teammate Yds', 'Teammate Y/R', 'Teammate TD',
       'Teammate 1D', 'Teammate Lng', 'Teammate Y/Tgt', 'Teammate R/G',
       'Teammate Y/G', 'Teammate Fmb', 'Teammate Year'],
      dtype='object')

In [91]:
teammate_receiving.rename(columns={'Teammate Player': 'Teammate'}, inplace=True)
teammate_rushing.rename(columns={'Teammate Player': 'Teammate'}, inplace=True)

In [93]:
teammate_receiving.to_csv('C:/Users/kevin/Downloads/Teammate Receiving Stats 2017-2022.csv', index=False)
teammate_rushing.to_csv('C:/Users/kevin/Downloads/Teammate Rushing Stats 2017-2022.csv', index=False)

The below code is a list of rookie running backs and receivers drafted since 2017. I had ChatGPT fill in most of the data and I filled in where necessary. The fields here that include the rookie's height, weight, draft pick. These will serve as predictors in our model in addition to the teammate's previous year stats once we merge those dataframes. The stats from the rookie will serve as the test data -- it will only be used to calculate the fantasy points they eventually achieved.

In [94]:
rookie_wr = pd.read_csv("C:/Users/kevin/Downloads/NFL Rookie Wide Receivers 2017-2023.csv")
rookie_rb = pd.read_csv("C:/Users/kevin/Downloads/NFL Rookie Running Backs 2017-2023.csv")

In [95]:
rookie_wr.rename(columns={'Name': 'Player'}, inplace=True)
rookie_rb.rename(columns={'Name': 'Player'}, inplace=True)

In [96]:
receiving.columns

Index(['Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'Tgt', 'Rec', 'Ctch%', 'Yds',
       'Y/R', 'TD', '1D', 'Lng', 'Y/Tgt', 'R/G', 'Y/G', 'Fmb', 'Year'],
      dtype='object')

In [97]:
temp = pd.merge(rookie_wr, receiving, on=['Player', 'Year'], how='left')

In [102]:
final_wr = pd.merge(temp, teammate_receiving, on=['Teammate', 'Teammate Year'], how='left')

In [112]:
temp = pd.merge(rookie_rb, rushing, on=['Player', 'Year'], how='left')

In [113]:
final_rb = pd.merge(temp, teammate_rushing, on=['Teammate', 'Teammate Year'], how='left')

Example of all the columns we have below.

In [117]:
final_wr.columns

Index(['Player', 'Team', 'Year', 'Height', 'Weight', '40 Time', 'Pick',
       'Teammate', 'Teammate Year', 'Tm', 'Age', 'Pos', 'G', 'GS', 'Tgt',
       'Rec', 'Ctch%', 'Yds', 'Y/R', 'TD', '1D', 'Lng', 'Y/Tgt', 'R/G', 'Y/G',
       'Fmb', 'Teammate Tm', 'Teammate Age', 'Teammate Pos', 'Teammate G',
       'Teammate GS', 'Teammate Tgt', 'Teammate Rec', 'Teammate Ctch%',
       'Teammate Yds', 'Teammate Y/R', 'Teammate TD', 'Teammate 1D',
       'Teammate Lng', 'Teammate Y/Tgt', 'Teammate R/G', 'Teammate Y/G',
       'Teammate Fmb'],
      dtype='object')

In [121]:
# Convert 'Age' and 'Teammate Age' columns to numeric type
final_wr['Age'] = pd.to_numeric(final_wr['Age'], errors='coerce')
final_wr['Teammate Age'] = pd.to_numeric(final_wr['Teammate Age'], errors='coerce')

# Calculate the average age (excluding null values)
average_rookiewr_age = final_wr['Age'].mean()
average_teammatewr_age = final_wr['Teammate Age'].mean()

# Fill null values in 'Age' and 'Teammate Age' with their respective average ages
final_wr['Age'].fillna(average_rookiewr_age, inplace=True)
final_wr['Teammate Age'].fillna(average_teammatewr_age, inplace=True)

In [122]:
# Convert 'Age' and 'Teammate Age' columns to numeric type
final_rb['Age'] = pd.to_numeric(final_rb['Age'], errors='coerce')
final_rb['Teammate Age'] = pd.to_numeric(final_rb['Teammate Age'], errors='coerce')

# Calculate the average age (excluding null values)
average_rookierb_age = final_rb['Age'].mean()
average_teammaterb_age = final_rb['Teammate Age'].mean()

# Fill null values in 'Age' and 'Teammate Age' with their respective average ages
final_rb['Age'].fillna(average_rookierb_age, inplace=True)
final_rb['Teammate Age'].fillna(average_teammaterb_age, inplace=True)

In [123]:
final_wr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221 entries, 0 to 220
Data columns (total 43 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Player          221 non-null    object 
 1   Team            221 non-null    object 
 2   Year            221 non-null    int64  
 3   Height          221 non-null    int64  
 4   Weight          221 non-null    int64  
 5   40 Time         221 non-null    float64
 6   Pick            221 non-null    int64  
 7   Teammate        221 non-null    object 
 8   Teammate Year   221 non-null    int64  
 9   Tm              148 non-null    object 
 10  Age             221 non-null    float64
 11  Pos             148 non-null    object 
 12  G               148 non-null    object 
 13  GS              148 non-null    object 
 14  Tgt             148 non-null    object 
 15  Rec             148 non-null    object 
 16  Ctch%           148 non-null    object 
 17  Yds             148 non-null    obj

In [124]:
final_wr = final_wr.apply(pd.to_numeric, errors='ignore')
final_rb = final_rb.apply(pd.to_numeric, errors='ignore')

In [128]:
final_wr.columns

Index(['Player', 'Team', 'Year', 'Height', 'Weight', '40 Time', 'Pick',
       'Teammate', 'Teammate Year', 'Tm', 'Age', 'Pos', 'G', 'GS', 'Tgt',
       'Rec', 'Ctch%', 'Yds', 'Y/R', 'TD', '1D', 'Lng', 'Y/Tgt', 'R/G', 'Y/G',
       'Fmb', 'Teammate Tm', 'Teammate Age', 'Teammate Pos', 'Teammate G',
       'Teammate GS', 'Teammate Tgt', 'Teammate Rec', 'Teammate Ctch%',
       'Teammate Yds', 'Teammate Y/R', 'Teammate TD', 'Teammate 1D',
       'Teammate Lng', 'Teammate Y/Tgt', 'Teammate R/G', 'Teammate Y/G',
       'Teammate Fmb'],
      dtype='object')

In [130]:
final_wr = final_wr.drop(['Ctch%','Teammate Ctch%'], axis=1)

The 2023 rookies will not have any end of season stats so we will need to fill them in with zeroes. They will be the test set as we predict their fantasy scores.

In [133]:
final_wr[['Player', 'G']].head()

,Player,G
0,Jaxon Smith-Njigba,NaN
1,Quentin Johnston,NaN
2,Zay Flowers,NaN
3,Jordan Addison,NaN
4,Jonathan Mingo,NaN


In [134]:
final_wr.fillna(0, inplace=True)

In [135]:
final_wr[['Player', 'G']].head()

,Player,G
0,Jaxon Smith-Njigba,0.0
1,Quentin Johnston,0.0
2,Zay Flowers,0.0
3,Jordan Addison,0.0
4,Jonathan Mingo,0.0


In [136]:
final_rb[['Player', 'G']].head()

,Player,G
0,Bijan Robinson,NaN
1,Jahmyr Gibbs,NaN
2,Zach Charbonnet,NaN
3,Kendre Miller,NaN
4,Tyjae Sparks,NaN


In [137]:
final_rb.fillna(0, inplace=True)

In [138]:
final_rb[['Player', 'G']].head()

,Player,G
0,Bijan Robinson,0.0
1,Jahmyr Gibbs,0.0
2,Zach Charbonnet,0.0
3,Kendre Miller,0.0
4,Tyjae Sparks,0.0


In [7]:
final_rb = pd.merge(final_rb, receiving, on=['Player', 'Year'], how='left')

In [10]:
final_rb = final_rb.drop(['Tm_y', 'Age_y', 'Pos_y', 'G_y', 'GS_y', 'Ctch%'], axis=1)

In [14]:
final_rb.fillna(0, inplace=True)

For running backs, they have a fumble column from the rushing data set and a fumble column from the receiving data set so I am combining them for simplicity. I do not have this issue with receivers because I did not pull in rushing data for them. While rushing data does add to a receiver's fantasy points, it has a such a small impact on PPR that I have kept it out for simplicity.

In [16]:
final_rb['Fumbles'] = final_rb['Rush_Fmb'] + final_rb['Fmb']

In [19]:
final_rb[['Player','Fumbles', 'Rush_Fmb', 'Fmb']][final_rb['Year'] == 2022].head(5)

,Player,Fumbles,Rush_Fmb,Fmb
17,Breece Hall,2.0,1.0,1.0
18,Kenneth Walker III,0.0,0.0,0.0
19,James Cook,2.0,1.0,1.0
20,Rachaad White,6.0,3.0,3.0
21,Tyrion Davis-Price,0.0,0.0,0.0


In [20]:
final_rb = final_rb.drop(['Rush_Fmb', 'Fmb'], axis=1)

In [22]:
final_rb.columns

Index(['Player', 'Team', 'Year', 'Height', 'Weight', '40 Time', 'Pick',
       'Teammate', 'Teammate Year', 'Tm_x', 'Age_x', 'Pos_x', 'G_x', 'GS_x',
       'Rush_Att', 'Rush_Yds', 'Rush_TD', 'Rush_1D', 'Rush_Lng', 'Rush_Y/A',
       'Rush_Y/G', 'Teammate Tm', 'Teammate Age', 'Teammate Pos', 'Teammate G',
       'Teammate GS', 'Teammate Rush_Att', 'Teammate Rush_Yds',
       'Teammate Rush_TD', 'Teammate Rush_1D', 'Teammate Rush_Lng',
       'Teammate Rush_Y/A', 'Teammate Rush_Y/G', 'Teammate Rush_Fmb', 'Tgt',
       'Rec', 'Yds', 'Y/R', 'TD', '1D', 'Lng', 'Y/Tgt', 'R/G', 'Y/G',
       'Fumbles'],
      dtype='object')

### Target Columns

The total fantasy points on the year and fantasy points per game are calculated based on the total stats for running backs and receivers. Those will be the target in our linear regression models.

In [23]:
final_rb['Total FP'] = ((final_rb['Rush_Yds'] * 0.1) + (final_rb['Rush_TD'] * 6) +
                        (final_rb['Rec']) + (final_rb['Yds'] * 0.1) + (final_rb['TD'] * 6) - (final_rb['Fumbles'] * 2))

In [24]:
final_rb['FPPG'] = final_rb['Total FP'] / final_rb['G_x']

In [26]:
final_rb[['Player','Rush_Yds', 'Rush_TD', 'Rec', 'Yds', 'TD', 'Fumbles', 'G_x', 'Total FP', 'FPPG']][final_rb['Year'] == 2022].head(5)

,Player,Rush_Yds,Rush_TD,Rec,Yds,TD,Fumbles,G_x,Total FP,FPPG
17,Breece Hall,463.0,4.0,19.0,218.0,1.0,2.0,7.0,113.1,16.157143
18,Kenneth Walker III,1050.0,9.0,27.0,165.0,0.0,0.0,15.0,202.5,13.500000
19,James Cook,507.0,2.0,21.0,180.0,1.0,2.0,16.0,103.7,6.481250
20,Rachaad White,481.0,1.0,50.0,290.0,2.0,6.0,17.0,133.1,7.829412
21,Tyrion Davis-Price,99.0,0.0,0.0,0.0,0.0,0.0,6.0,9.9,1.650000


In [27]:
final_wr.columns

Index(['Player', 'Team', 'Year', 'Height', 'Weight', '40 Time', 'Pick',
       'Teammate', 'Teammate Year', 'Tm', 'Age', 'Pos', 'G', 'GS', 'Tgt',
       'Rec', 'Yds', 'Y/R', 'TD', '1D', 'Lng', 'Y/Tgt', 'R/G', 'Y/G', 'Fmb',
       'Teammate Tm', 'Teammate Age', 'Teammate Pos', 'Teammate G',
       'Teammate GS', 'Teammate Tgt', 'Teammate Rec', 'Teammate Yds',
       'Teammate Y/R', 'Teammate TD', 'Teammate 1D', 'Teammate Lng',
       'Teammate Y/Tgt', 'Teammate R/G', 'Teammate Y/G', 'Teammate Fmb'],
      dtype='object')

In [28]:
final_wr['Total FP'] = ((final_wr['Rec']) + (final_wr['Yds'] * 0.1) + (final_wr['TD'] * 6) - (final_wr['Fmb'] * 2))
final_wr['FPPG'] = final_wr['Total FP'] / final_wr['G']

In [30]:
final_wr[['Player', 'Rec', 'Yds', 'TD', 'Fmb', 'G', 'Total FP', 'FPPG']][final_wr['Year'] == 2022].head(5)

,Player,Rec,Yds,TD,Fmb,G,Total FP,FPPG
32,Drake London,72.0,866.0,4.0,3.0,17.0,176.6,10.388235
33,Garrett Wilson,83.0,1103.0,4.0,2.0,17.0,213.3,12.547059
34,Chris Olave,72.0,1042.0,4.0,2.0,15.0,196.2,13.080000
35,Jameson Williams,1.0,41.0,1.0,0.0,6.0,11.1,1.850000
36,Jahan Dotson,35.0,523.0,7.0,0.0,12.0,129.3,10.775000


In [31]:
final_wr.to_csv('C:/Users/kevin/Downloads/Full Receiving Model 2017-2022.csv', index=False)
final_rb.to_csv('C:/Users/kevin/Downloads/Full Rushing Model 2017-2022.csv', index=False)